# CASE 1: Static HWP Cycles - Intensities Matrix Inversion (1% Noise)

In [1]:
import numpy as np
import sys
sys.path.insert(0, '../python/')
import angles
import instrument_mm
import stokes

# Define observer parameters
observer_latitude = 20.0  # Latitude of Mauna Kea in degrees
observer_longitude = -155.5  # Longitude of Mauna Kea in degrees

# Define target parameters
targets = [
    {"ra": 45.0, "dec": -50.0},  # RA 3h (45°), Dec -50°
    {"ra": 45.0, "dec": -20.0},  # RA 3h (45°), Dec -20°
    {"ra": 45.0, "dec": 15.0},   # RA 3h (45°), Dec 15°
]

# Polarization levels to inject (0.01%, 0.1%, and 1%)
polarization_levels = [0.0001, 0.001, 0.01]  # Convert percentages to decimal form

# Observation date and time
observation_date = "2460601.5"  # JD for 10/31/2024
observation_time = "14:00:00"   # UT time to get peak at midnight HST

# Run the simulation
for target in targets:
    print("*************************")
    print(f"Target RA: {target['ra']}°, Dec: {target['dec']}°")
    
    for pol in polarization_levels:
        # Inject Q and U polarization levels
        Q_injected = pol
        U_injected = 0
        injected_stokes = np.array([1, Q_injected, U_injected, 0])
        
        # Calculate the parallactic angle
        pa = angles.calculate_parallactic_angle(
            ra=target["ra"], 
            dec=target["dec"], 
            ut=observation_time, 
            jd_str=observation_date, 
            observer_latitude=observer_latitude, 
            observer_longitude=observer_longitude
        )
        
        # Calculate the altitude for each target at the given date and time
        H = angles.calculate_hour_angle(
            ra=target["ra"], 
            observer_longitude=observer_longitude, 
            ut=observation_time, 
            jd_str=observation_date
        )
        altitude = angles.calculate_altitude(
            phi=observer_latitude, 
            delta=target["dec"], 
            H=H
        )
        
        # Calculate the inverted input Stokes vector S_in using the new function
        S_in_retrieved = instrument_mm.calculate_input_Q_U_intensities_matrix_inversion(
            Q=Q_injected, 
            U=U_injected, 
            pa=pa, 
            altitude=altitude,
            noise_percentage = 0.1
        )
        
        # Calculate the absolute difference between injected and retrieved Stokes vectors
        absolute_difference = np.abs(injected_stokes - S_in_retrieved)
        
        # Output results
        print(f"Injected Stokes Vector: {injected_stokes}")
        print(f"Retrieved S_in: {S_in_retrieved}")
        print(f"Absolute Difference: {absolute_difference}\n")


*************************
Target RA: 45.0°, Dec: -50.0°
Injected Stokes Vector: [1.e+00 1.e-04 0.e+00 0.e+00]
Retrieved S_in: [1.00038170e+00 2.99459729e-04 1.36569893e-03 4.38395207e-03]
Absolute Difference: [0.0003817  0.00019946 0.0013657  0.00438395]

Injected Stokes Vector: [1.    0.001 0.    0.   ]
Retrieved S_in: [1.0006102  0.0019698  0.00217907 0.00695351]
Absolute Difference: [0.0006102  0.0009698  0.00217907 0.00695351]

Injected Stokes Vector: [1.   0.01 0.   0.  ]
Retrieved S_in: [0.99994311 0.01244737 0.00767712 0.02703746]
Absolute Difference: [5.68881634e-05 2.44736976e-03 7.67712383e-03 2.70374615e-02]

*************************
Target RA: 45.0°, Dec: -20.0°
Injected Stokes Vector: [1.e+00 1.e-04 0.e+00 0.e+00]
Retrieved S_in: [ 9.99787394e-01 -1.45541976e-05  1.60092032e-03  5.32587592e-03]
Absolute Difference: [0.00021261 0.00011455 0.00160092 0.00532588]

Injected Stokes Vector: [1.    0.001 0.    0.   ]
Retrieved S_in: [1.00069952e+00 6.28304358e-05 1.18120774e-03 

# CASE 2: Q and U Observing Sequence - Intensities Matrix Inversion (1% noise)

In [2]:
import numpy as np
import sys
sys.path.insert(0, '../python/')
import angles
import instrument_mm
import stokes

# Define observer parameters
observer_latitude = 20.0  # Latitude of Mauna Kea in degrees
observer_longitude = -155.5  # Longitude of Mauna Kea in degrees

# Define target parameters
targets = [
    {"ra": 45.0, "dec": -50.0},  # RA 3h (45°), Dec -50°
    {"ra": 45.0, "dec": -20.0},  # RA 3h (45°), Dec -20°
    {"ra": 45.0, "dec": 15.0},   # RA 3h (45°), Dec 15°
]

# Polarization levels to inject (0.01%, 0.1%, and 1%)
polarization_levels = [0.0001, 0.001, 0.01]  # Convert percentages to decimal form

# Observation date and time
observation_date = "2460601.5"  # JD for 10/31/2024
observation_time = "14:00:00"   # UT time to get peak at midnight HST
t_int = 60  # Integration time in seconds (modifiable variable)

# Run the simulation
for target in targets:
    print("*************************")
    print(f"Target RA: {target['ra']}°, Dec: {target['dec']}°")
    
    for pol in polarization_levels:
        # Inject Q and U polarization levels
        Q_injected = pol
        U_injected = 0
        injected_stokes = np.array([1, Q_injected, U_injected, 0])
        
        # Use the modified function with dynamic PA and altitude
        S_in_retrieved = instrument_mm.calculate_input_Q_U_observing_sequence_matrix_inversion(
            ra=target["ra"], 
            dec=target["dec"], 
            observer_latitude=observer_latitude, 
            observer_longitude=observer_longitude, 
            jd_str=observation_date, 
            ut_start=observation_time, 
            t_int=t_int, 
            Q=Q_injected, 
            U=U_injected, 
            delta_HWP=0.5,
            noise_percentage = 0.1
        )
        
        # Calculate the absolute difference between injected and retrieved Stokes vectors
        absolute_difference = np.abs(injected_stokes - S_in_retrieved)
        
        # Output results
        print(f"Injected Stokes Vector: {injected_stokes}")
        print(f"Retrieved S_in: {S_in_retrieved}")
        print(f"Absolute Difference: {absolute_difference}\n")


*************************
Target RA: 45.0°, Dec: -50.0°
Injected Stokes Vector: [1.e+00 1.e-04 0.e+00 0.e+00]
Retrieved S_in: [ 1.00020415e+00 -5.73296790e-04 -2.67685959e-03 -7.77889372e-03]
Absolute Difference: [0.00020415 0.0006733  0.00267686 0.00777889]

Injected Stokes Vector: [1.    0.001 0.    0.   ]
Retrieved S_in: [0.99963806 0.0013913  0.00106856 0.00230714]
Absolute Difference: [0.00036194 0.0003913  0.00106856 0.00230714]

Injected Stokes Vector: [1.   0.01 0.   0.  ]
Retrieved S_in: [ 9.99836428e-01  9.14102934e-03 -2.93170201e-04 -3.00577697e-03]
Absolute Difference: [0.00016357 0.00085897 0.00029317 0.00300578]

*************************
Target RA: 45.0°, Dec: -20.0°
Injected Stokes Vector: [1.e+00 1.e-04 0.e+00 0.e+00]
Retrieved S_in: [9.99965463e-01 5.84003231e-04 1.96434604e-03 6.14171620e-03]
Absolute Difference: [3.45373817e-05 4.84003231e-04 1.96434604e-03 6.14171620e-03]

Injected Stokes Vector: [1.    0.001 0.    0.   ]
Retrieved S_in: [0.99959293 0.00140894 0.0